## Segmenting and Clustering Part 1

In [1]:
import requests
import lxml.html as lh
import pandas as pd

Establish connection to the url , parse the required contents and store them 

In [4]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')
col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighborhood
"


Storing the data in DataFrame

In [21]:
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    if len(T)!=3:
        break
    i=0
    for t in T.iterchildren():
        data=t.text_content() 
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1
        

        
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [23]:

df.columns = ['Borough', 'Neighbourhood','Postcode']

cols = df.columns.tolist()
df.head()

,Borough,Neighbourhood,Postcode
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [24]:
df = df.replace('\n',' ', regex=True)
df.head()

,Borough,Neighbourhood,Postcode
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Dropping all cells with a borough that is Not assigned

In [25]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)
df = df.reset_index(drop=True)
df.head(10)

,Borough,Neighbourhood,Postcode
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
8,M8A,Not assigned,Not assigned
9,M9A,Downtown Toronto,Queen's Park


Groupby Function 

In [26]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(10)

,Postcode,Borough,Neighbourhood
0,Adelaide,M5H,"Downtown Toronto,Downtown Toronto,Downtown Tor..."
1,Agincourt,M1S,"Scarborough,Scarborough,Scarborough,Scarborough"
2,Agincourt North,M1V,"Scarborough,Scarborough,Scarborough,Scarborough"
3,Albion Gardens,M9V,"Etobicoke,Etobicoke,Etobicoke,Etobicoke"
4,Alderwood,M8W,"Etobicoke,Etobicoke,Etobicoke,Etobicoke"
5,Bathurst Manor,M3H,"North York,North York,North York,North York"
6,Bathurst Quay,M5V,"Downtown Toronto,Downtown Toronto,Downtown Tor..."
7,Bayview Village,M2K,"North York,North York,North York,North York"
8,Beaumond Heights,M9V,"Etobicoke,Etobicoke,Etobicoke,Etobicoke"
9,Bedford Park,M5M,"North York,North York,North York,North York"






















#### Removing any space in the start of the string

In [20]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

KeyError: 'Neighbourhood'

#### Assigning Borough values to the Neignbourhood where vlaue is "Not assigned"

In [14]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [15]:
# Check if the Neighbourhood for Queen's Park changed 
df[df['Borough'] == 'Queen\'s Park']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [16]:
# Check the shape of the data frame
df.shape

(103, 3)

#### Save this file to a csv

In [17]:
df.to_csv(r'df_can.csv')